In [1]:
%pip install accelerate -U
%pip install transformers
%pip install datasets
%pip install seqeval

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: transformers in c:\users\obe\appdata\local\programs\python\python311\lib\site-packages (4.38.2)




[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from transformers import AutoTokenizer
from transformers import AutoModelForTokenClassification, Trainer
from datasets import Dataset
import numpy as np
import pandas as pd
import pickle
from huggingface_hub import login
import os
from datasets import load_metric

## Testing

! IMPORTANT 
To run the testing you need to have a trained model that is already saved as well as a pickle of the label_indices from training.

Specify the path of the testing set and the path for saving the file with predictions

In [3]:
# The only thing you have to change is the language of the model you're working with
model_language = "serbian"





# List of test data file paths
test_data_files = [r"data\zh_pud-ud-test.iob2",
                   r"data\da_ddt-ud-test.iob2",
                   r"data\en_ewt-ud-test.iob2",
                   r"data\de_pud-ud-test.iob2",
                   r"data\pt_pud-ud-test.iob2",
                   r"data\ru_pud-ud-test.iob2",
                   r"data\sk_snk-ud-test.iob2",
                   r"data\sv_pud-ud-test.iob2",
                   r"data\sr_test.iob2",
                   r"data\hr_test.iob2",

]


# where should we pull the model from
hub_folder = f"annamariagnat/NEW_trained_{model_language}"

# specify a path to your label_indices
labels = f"01_label_indices/NEW_labels_{model_language}.pkl"

label_all_tokens = True # dw about it

In [4]:
# repeated the functions needed to load in and format the test dataset

#reading in data as a dataframe
def read_iob2_file(path):
    data = []
    current_words = []
    current_tags = []

    for line in open(path, encoding='utf-8'):
        line = line.strip()

        if line:
            if line[0] == '#':
                continue
            tok = line.split('\t')

            current_words.append(tok[1])
            current_tags.append(tok[2])
        else:
            if current_words:
                data.append((current_words, current_tags))
            current_words = []
            current_tags = []

    if current_tags != []:
        data.append((current_words, current_tags))

    df = pd.DataFrame(data, columns=['words', 'tags'])
    df['id'] = df.index
    df = df[['id', 'words', 'tags']]
    
    return df

# tokenizing the labels
def tokenize_and_align_labels(dataset, word_column, tag_column, tokenizer):
    tokenized_inputs = tokenizer(dataset[word_column].tolist(), truncation=True , is_split_into_words=True, padding = True)

    labels = []
    for i, label in enumerate(dataset[tag_column]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs.data


class Vocab():
    def __init__(self, pad_unk='<PAD>'):
        self.pad_unk = pad_unk
        self.word2idx = {}
        self.idx2word = []

    def getIdx(self, word, add=False):
        if word is None or word == self.pad_unk:
            return None
        if word not in self.word2idx:
            if add:
                idx = len(self.idx2word)
                self.word2idx[word] = idx
                self.idx2word.append(word)
                return idx
            else:
                return None
        return self.word2idx[word]

    def getWord(self, idx):
        return self.idx2word[idx]

In [5]:
# Password for the hub (Anna)

login()

In [6]:
loaded_model = AutoModelForTokenClassification.from_pretrained(hub_folder)
trainer = Trainer(model = loaded_model)
tokenizer = tokenizer = AutoTokenizer.from_pretrained(hub_folder)

config.json:   0%|          | 0.00/958 [00:00<?, ?B/s]

c:\Users\obe\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\obe\.cache\huggingface\hub\models--annamariagnat--NEW_trained_serbian. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/539M [00:00<?, ?B/s]

c:\Users\obe\AppData\Local\Programs\Python\Python311\Lib\site-packages\accelerate\accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [7]:
# Load label_indices dictionary from the pickle file
with open(labels, 'rb') as f:
    label_indices = pickle.load(f)

label_list = label_indices.idx2word

In [10]:
metric = load_metric("seqeval")
for test_path in test_data_files:
    test_data = read_iob2_file(test_path)

    test_data['tag_idx'] = test_data['tags'].apply(lambda x: [label_indices.word2idx[tag] for tag in x])

    # tokenize the data
    tokenized_test_data = tokenize_and_align_labels(test_data, "words", "tag_idx", tokenizer)


    # turning the data into datasetdicts, to make them compatible with the trainer (otherwise they can't be indexed)

    test_dataset = Dataset.from_dict({
        'id': range(len(tokenized_test_data['input_ids'])),
        'input_ids': tokenized_test_data['input_ids'],
        'attention_mask': tokenized_test_data['attention_mask'],
        'labels': tokenized_test_data['labels']
    })

    # evaluating using test data
    test_dataset_new = Dataset.from_dict({
        'input_ids': test_dataset['input_ids'],
        'attention_mask': test_dataset['attention_mask'],
        'labels': test_dataset['labels']
    })

    predictions, labels, _ = trainer.predict(test_dataset_new)
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    print(f"Results for {test_path}:")
    print(results)

c:\Users\obe\AppData\Local\Programs\Python\Python311\Lib\site-packages\datasets\load.py:756: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


Results for data\zh_pud-ud-test.iob2:
{'LOC': {'precision': 0.6352288488210819, 'recall': 0.6325966850828729, 'f1': 0.6339100346020761, 'number': 1448}, 'ORG': {'precision': 0.35610766045548653, 'recall': 0.45989304812834225, 'f1': 0.40140023337222874, 'number': 374}, 'PER': {'precision': 0.7288267288267288, 'recall': 0.7797173732335827, 'f1': 0.753413654618474, 'number': 1203}, 'overall_precision': 0.6307596513075965, 'overall_recall': 0.6697520661157025, 'overall_f1': 0.6496713163379829, 'overall_accuracy': 0.945479395073833}


  0%|          | 0/71 [00:00<?, ?it/s]

Results for data\da_ddt-ud-test.iob2:
{'LOC': {'precision': 0.591743119266055, 'recall': 0.6789473684210526, 'f1': 0.6323529411764707, 'number': 190}, 'ORG': {'precision': 0.5531197301854974, 'recall': 0.6393762183235867, 'f1': 0.5931283905967449, 'number': 513}, 'PER': {'precision': 0.7639751552795031, 'recall': 0.7299703264094956, 'f1': 0.7465857359635812, 'number': 337}, 'overall_precision': 0.6204766107678729, 'overall_recall': 0.6759615384615385, 'overall_f1': 0.6470317533364013, 'overall_accuracy': 0.9581347417235605}


  0%|          | 0/260 [00:00<?, ?it/s]

...

Results for data\sv_pud-ud-test.iob2:
{'LOC': {'precision': 0.7827191867852605, 'recall': 0.7503045066991474, 'f1': 0.7661691542288558, 'number': 821}, 'ORG': {'precision': 0.4411764705882353, 'recall': 0.5603448275862069, 'f1': 0.49367088607594933, 'number': 348}, 'PER': {'precision': 0.8188736681887366, 'recall': 0.7752161383285303, 'f1': 0.7964470762398224, 'number': 694}, 'overall_precision': 0.7152704135737009, 'overall_recall': 0.7241009125067096, 'overall_f1': 0.7196585756201652, 'overall_accuracy': 0.9724307490575316}


  0%|          | 0/65 [00:00<?, ?it/s]

Results for data\sr_test.iob2:
{'LOC': {'precision': 0.9432624113475178, 'recall': 0.9736456808199122, 'f1': 0.9582132564841499, 'number': 683}, 'ORG': {'precision': 0.8469387755102041, 'recall': 0.8615916955017301, 'f1': 0.8542024013722127, 'number': 578}, 'PER': {'precision': 0.9211538461538461, 'recall': 0.9410609037328095, 'f1': 0.9310009718172982, 'number': 509}, 'overall_precision': 0.9056811913954771, 'overall_recall': 0.927683615819209, 'overall_f1': 0.9165503767792352, 'overall_accuracy': 0.9842645154865053}


  0%|          | 0/142 [00:00<?, ?it/s]

Results for data\hr_test.iob2:
{'LOC': {'precision': 0.902135231316726, 'recall': 0.9094170403587444, 'f1': 0.9057615006699419, 'number': 1115}, 'ORG': {'precision': 0.7407087294727744, 'recall': 0.825626204238921, 'f1': 0.780865603644647, 'number': 1038}, 'PER': {'precision': 0.886150234741784, 'recall': 0.9009546539379475, 'f1': 0.893491124260355, 'number': 838}, 'overall_precision': 0.8381742738589212, 'overall_recall': 0.8779672350384486, 'overall_f1': 0.8576094056172435, 'overall_accuracy': 0.9797745511898687}


In [18]:
# functions needed for the testing phase

def un_tok_labs(list_of_labels, list_of_words):
    tokenized_inputs = tokenizer(list_of_words, is_split_into_words=True)
    labels = []
    for i, label in enumerate(list_of_labels):
        
        label_copy = label.copy()  # Create a copy of the label list
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:  # Only label the first token of a given word.
                continue
            elif word_idx == previous_word_idx:
                label_copy.pop(word_idx)
                continue
            else:
                label_ids.append(label_copy[word_idx])
            previous_word_idx = word_idx 
        labels.append(label_ids)
    return labels



def read_list_of_words(path):
    """
    read in iob2 file
    
    :param path: path to read from
    :returns: list with sequences of words for each sentence
    """
    data = []
    current_words = []

    for line in open(path, encoding='utf-8'):
        line = line.strip()

        if line:
            if line[0] == '#':
                continue # skip comments
            tok = line.split('\t')

            current_words.append(tok[1])
        else:
            if current_words:  # skip empty lines
                data.append(current_words)
            current_words = []

    # Check for the last sentence
    if current_words:
        data.append(current_words)
    
    return data



def save_preds(filename, tok, untok_labs):
    file_path = "data/" + filename
    
    if os.path.exists(file_path):
        print(f"The file {filename} already exists. Skipping saving it.")
        return  # Skip saving the file if it already exists

    with open(file_path, "w", encoding="utf-8") as f: 
        for t, l in zip(tok, untok_labs): 
            for i in range(len(t)): 
                f.write(f"{i+1}\t{t[i]}\t{l[i]}\n")

            f.write("\n")
    
    return "File has been saved"

In [19]:
for test_path in test_data_files:
    testing_language_code = os.path.basename(test_path).split('_')[0]  # Extracting the language code from the file name
    output_filename = f"NEW_{model_language}_on_{testing_language_code}_predictions"

    test_data = read_iob2_file(test_path)
    test_data['tag_idx'] = test_data['tags'].apply(lambda x: [label_indices.word2idx[tag] for tag in x])
    tokenized_test_data = tokenize_and_align_labels(test_data, "words", "tag_idx", tokenizer)

    test_dataset = Dataset.from_dict({
        'id': range(len(tokenized_test_data['input_ids'])),
        'input_ids': tokenized_test_data['input_ids'],
        'attention_mask': tokenized_test_data['attention_mask'],
        'labels': tokenized_test_data['labels']
    })

    test_dataset_new = Dataset.from_dict({
        'input_ids': test_dataset['input_ids'],
        'attention_mask': test_dataset['attention_mask'],
        'labels': test_dataset['labels']
    })

    # Perform testing phase
    predictions, labels, _ = trainer.predict(test_dataset_new)
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    test_words = read_list_of_words(test_path)
    untok_labs = un_tok_labs(true_predictions, test_words)

    # Save predictions
    save_preds(output_filename, test_words, untok_labs)


  0%|          | 0/125 [00:00<?, ?it/s]

  0%|          | 0/71 [00:00<?, ?it/s]

  0%|          | 0/260 [00:00<?, ?it/s]

  0%|          | 0/125 [00:00<?, ?it/s]

  0%|          | 0/125 [00:00<?, ?it/s]

  0%|          | 0/125 [00:00<?, ?it/s]

  0%|          | 0/133 [00:00<?, ?it/s]

  0%|          | 0/125 [00:00<?, ?it/s]

  0%|          | 0/65 [00:00<?, ?it/s]

  0%|          | 0/142 [00:00<?, ?it/s]